In [1]:
import spacy
import pandas as pd
import numpy as np
import time
from statistics import mean 
from multiprocessing import Pool
import pickle
from spacy.tokens import DocBin

pd.options.display.max_columns = 500
pd.options.display.max_rows = 500

nlp = spacy.load("pt_core_news_sm")

In [2]:
def tokeniza(df):
    df["Docs"] = df["0"].apply(lambda x: nlp(x)).to_list()
    return df


def parallelize_dataframe(df, func, n_cores=12):
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.terminate()
    pool.join()
    return df

def compara(df):
    df_vazio=pd.DataFrame(columns=["id","similarity"])
    #itera mercado
    for i in range(len(df)):
        doc_mercado = df["Docs"].iloc[i]
        similarity_sum=0
        #iteraportfolio
        for j in range(len(df_port_dupl)):
            doc_portfolio = df_port_dupl["Docs"][j]
            similarity_sum+=(doc_mercado.similarity(doc_portfolio)*dici_port[df_port_dupl["0"][j]])
            semelhanca_media = similarity_sum/len(df_port)
        df_vazio = df_vazio.append({'id':df['id'].iloc[i],'similarity':semelhanca_media},ignore_index=True)
    return df_vazio

def parallelize_compare(df, func, n_cores=12):
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split)).reset_index(drop=True)
    pool.close()
    pool.terminate()
    pool.join()
    return df

In [3]:
df = pd.read_csv("MERCADO_FINAL.csv")
port = pd.read_csv("estaticos_portfolio1.csv",index_col=0)
df_port = df[df['id'].isin(port['id'])].reset_index(drop=True)

In [4]:
print(len(df))
df

462298


,id,0
0,a6984c3ae395090e3bee8ad63c3758b110de096d5d8195...,sociedade empresaria limitada entidades empres...
1,6178f41ade1365e44bc2c46654c2c8c0eaae27dcb476c4...,empresario individual outros servicos alojamen...
2,4a7e5069a397f12fdd7fd57111d6dc5d3ba558958efc02...,empresario individual outros transporte armaze...
3,3348900fe63216a439d2e5238c79ddd46ede454df7b9d8...,empresario individual outros servicos diversos...
4,1f9bcabc9d3173c1fe769899e4fac14b053037b953a1e4...,empresario individual outros servicos profissi...
...,...,...
462293,469ee5fc984d0bb60aa2f40cda4d003d7c9b963d149ed3...,empresario individual outros comercio reparaca...
462294,656f496e7e2fad708f0022d8ad97d2cb9344e839a5f3cb...,empresario individual outros servicos administ...
462295,9b933a5cf2149d1338f00fb459d34a16b9b0f2e8d3e8e1...,associacao privada entidades fins lucrativos s...
462296,84db0f17467cf4f02439e01ade57150b6ac59d9dae53ed...,sociedade empresaria limitada entidades empres...


In [ ]:
df_dupl = df.drop_duplicates(subset='0', keep="last").reset_index(drop=True)
df_port_dupl = df_port.drop_duplicates(subset='0', keep="last").reset_index(drop=True)
dici_dupl = df['0'].value_counts().to_dict()
dici_port = df_port['0'].value_counts().to_dict()

start_time = time.time()
df_dupl = parallelize_dataframe(df_dupl,tokeniza)
print("--- %s seconds ---" % (time.time() - start_time))

start_time = time.time()
df_port_dupl = parallelize_dataframe(df_port_dupl,tokeniza)
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
len(df_dupl),len(d)

In [6]:
%%capture
start_time = time.time()
#itera mercado
df_vazio = parallelize_compare(df_dupl,compara)
tempo_total = (time.time() - start_time)
print("--- %s seconds ---" % (time.time() - start_time))

In [7]:
df_vazio

,id,similarity
0,6c45aabb849a760a3a7107c7a61262642f683feb8912a4...,0.893553
1,764bb8648b96fd6459fa2f2237c22f3d6b8c129a814e33...,0.875062
2,a81c8e1d49c50f9ee7e5e7b25cd2223eba13ffcff402fe...,0.823826
3,128ad43b6d43ab63679620906fc4bcb733f7561513967d...,0.869605
4,a820394924e9d954f40431b995072bd5c4b8b7b1239d59...,0.825831
...,...,...
6827,469ee5fc984d0bb60aa2f40cda4d003d7c9b963d149ed3...,0.912193
6828,656f496e7e2fad708f0022d8ad97d2cb9344e839a5f3cb...,0.904166
6829,9b933a5cf2149d1338f00fb459d34a16b9b0f2e8d3e8e1...,0.858641
6830,84db0f17467cf4f02439e01ade57150b6ac59d9dae53ed...,0.924186


In [8]:
df_final = df_dupl.merge(df_vazio,on="id")[["id","0","similarity"]]
df_final.head(5)

,id,0,similarity
0,6c45aabb849a760a3a7107c7a61262642f683feb8912a4...,cooperativa entidades empresariais bens consum...,0.893553
1,764bb8648b96fd6459fa2f2237c22f3d6b8c129a814e33...,consorcio sociedades entidades empresariais in...,0.875062
2,a81c8e1d49c50f9ee7e5e7b25cd2223eba13ffcff402fe...,sociedade empresaria limitada entidades empres...,0.823826
3,128ad43b6d43ab63679620906fc4bcb733f7561513967d...,orgao publico poder executivo municipal admini...,0.869605
4,a820394924e9d954f40431b995072bd5c4b8b7b1239d59...,empresa individual responsabilidade limitada n...,0.825831


In [9]:
# inner join
df_final_mesmo = df.merge(df_final[["0","similarity"]],on="0")[["id","similarity"]]
# remove ids that are in portfolio, so that it not recommends any portfolio id
df_final_mesmo = df_final_mesmo[~df_final_mesmo['id'].isin(port["id"])]
df_final_mesmo.to_csv("Similar_port_1.csv",index=False)